In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
import food_tool as T
import numpy as np

# import tools.image_gen_extended as T

n_classes = 101
#keras 中没有这个方法，需要自己实现如何数据增强
#作者这里重写了ImageDataGenerator类，使用自己的数据增强方法，但是作者原文中使用的datagen.flow方法因为内存太小
#没办法用，所以只能使用给出的datagen.flow_from_directory方法，这个方法目前还不能random crop，而且作者给出的
#方法中有bug，我已经进行了修改。

# this is the augmentation configuration we will use for training
train_datagen = T.ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False, # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    seed=2147483647,
    fill_mode='reflect')
train_datagen.config['random_crop_size'] = (224, 224)
train_datagen.set_pipeline([T.random_transform, T.random_crop, T.preprocess_input])
train_generator = train_datagen.flow_from_directory('./food-101/train_split', batch_size=64,  target_size=(224, 224))
# validation 
val_datagen = T.ImageDataGenerator(seed=2147483647)
val_datagen.config['random_crop_size'] = (224, 224)
val_datagen.set_pipeline([T.random_transform, T.random_crop, T.preprocess_input])
val_generator = val_datagen.flow_from_directory('./food-101/val_split', batch_size=64, target_size=(224, 224))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
#training

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

K.clear_session()

base_model = MobileNet(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)),input_shape=(224, 224,3),classes=101)
# base_model.summary()
x = base_model.output
x = AveragePooling2D(pool_size=(7, 7))(x)
x = Dropout(.4)(x)
x = Flatten()(x)
predictions = Dense(n_classes, init='glorot_uniform', W_regularizer=l2(.0005), activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

opt = SGD(lr=.01, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='model4.{epoch:02d}.hdf5', save_best_only=True)
csv_logger = CSVLogger('./food_mobilenet.log')

def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004
lr_scheduler = LearningRateScheduler(schedule)
model.fit_generator(generator=train_generator,
                    steps_per_epoch=30,
                    epochs=100,  
                    validation_data=val_generator,
                    validation_steps=10,
                    verbose=1,
                    callbacks = [lr_scheduler, csv_logger, checkpointer])
# model.fit_generator(train_generator,
#                     validation_data=val_generator,
#                     nb_val_samples=64,
#                     samples_per_epoch=64,
#                     nb_epoch=32,
#                     verbose=1,
#                     callbacks=[lr_scheduler, csv_logger, checkpointer])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(101, activation="softmax", kernel_initializer="glorot_uniform", kernel_regularizer=<keras.reg...)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/100
30/30 [==============================] - 2647s 88s/step - loss: 5.0629 - acc: 0.0547 - val_loss: 4.9467 - val_acc: 0.1031
Epoch 2/100
30/30 [==============================] - 2604s 87s/step - loss: 4.1433 - acc: 0.1375 - val_loss: 5.1003 - val_acc: 0.0953
Epoch 3/100
30/30 [==============================] - 2602s 87s/step - loss: 3.7049 - acc: 0.1870 - val_loss: 4.3581 - val_acc: 0.1406
Epoch 4/100
30/30 [==============================] - 2733s 91s/step - loss: 3.4229 - acc: 0.2281 - val_loss: 4.3137 - val_acc: 0.1484
Epoch 5/100
30/30 [==============================] - 2672s 89s/step - loss: 3.1187 - acc: 0.2594 - val_loss: 3.4580 - val_acc: 0.2203
Epoch 6/100
 4/30 [===>..........................] - ETA: 33:34 - loss: 2.9993 - acc: 0.2852